## Climate Analysis using SQLAlchemy - Database Engineering

In [1]:
# Dependencies
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, ForeignKey, Integer, String, Numeric, Text, Float
from sqlalchemy.orm import Session

### Pull in CSVs and format data to be ORM table-ready

In [2]:
# Read weather data
df_measurements = pd.read_csv("Resources/clean_hawaii_measurements.csv")
df_stations = pd.read_csv("Resources/hawaii_stations.csv")
df_measurements.head()

,id,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.00,73


In [3]:
# Create lists from dataframes (to_dict() cleans out DataFrame metadata)
#  Ref: http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
measurements_data = df_measurements.to_dict(orient='records')
stations_data = df_stations.to_dict(orient='records')

In [4]:
# Make sure data looks good
#print(stations_data)
print(measurements_data)

[{'id': 0, 'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'id': 1, 'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'id': 2, 'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'id': 3, 'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'id': 4, 'station': 'USC00519397', 'date': '2010-01-06', 'prcp': 0.0, 'tobs': 73}, {'id': 5, 'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}, {'id': 6, 'station': 'USC00519397', 'date': '2010-01-08', 'prcp': 0.0, 'tobs': 64}, {'id': 7, 'station': 'USC00519397', 'date': '2010-01-09', 'prcp': 0.0, 'tobs': 68}, {'id': 8, 'station': 'USC00519397', 'date': '2010-01-10', 'prcp': 0.0, 'tobs': 73}, {'id': 9, 'station': 'USC00519397', 'date': '2010-01-11', 'prcp': 0.01, 'tobs': 64}, {'id': 10, 'station': 'USC00519397', 'date': '2010-01-12', 'prcp': 0.0, 'tobs': 61}, {'id': 11, 'station': 'USC00519397', 'date': '2010-01-14', 'prcp': 0.0,

### Create database and table structure

In [5]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [7]:
# Use "declarative_base" from SQLAlchemy to model the two weather tables as an ORM classes. Specify types for each column.
Base = declarative_base()

# Station table
class Station(Base):
    __tablename__ = 'station'

    station = Column(String(12), primary_key=True)
    name = Column(String(100))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
# Measurement table
class Measurement(Base):
    __tablename__ = 'measurement'

    id = Column(Integer, primary_key=True)
    station = Column(String(12), ForeignKey("Station.station"))
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [9]:
# Create the table structure in the database
Base.metadata.create_all(engine)

In [10]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
# Save the reference to the tables as a variable. Needed to bulk upload data, rather than add through individual objects
measurements_table = sqlalchemy.Table('measurement', metadata, autoload=True)
stations_table = sqlalchemy.Table('station', metadata, autoload=True)

In [12]:
from sqlalchemy import func

# Insert the data, as long as the tables do not already have data in them (if this has been run before)
count_measurements = session.query(func.count(Measurement.id)).first()
count_stations = session.query(func.count(Station.station)).first()

if count_measurements[0] == 0:
    conn.execute(measurements_table.insert(), measurements_data)
 
if count_stations[0] == 0:
    conn.execute(stations_table.insert(), stations_data)

In [13]:
# Connect to database and run test query against table
session = Session(bind=engine)
results = session.query(Measurement).limit(10)

for row in results:
    print(row.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000000000A020F28>, 'tobs': 65.0, 'date': '2010-01-01', 'id': 0, 'prcp': 0.08, 'station': 'USC00519397'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000000000A020F98>, 'tobs': 63.0, 'date': '2010-01-02', 'id': 1, 'prcp': 0.0, 'station': 'USC00519397'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000000000A03B048>, 'tobs': 74.0, 'date': '2010-01-03', 'id': 2, 'prcp': 0.0, 'station': 'USC00519397'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000000000A03B0B8>, 'tobs': 76.0, 'date': '2010-01-04', 'id': 3, 'prcp': 0.0, 'station': 'USC00519397'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000000000A03B128>, 'tobs': 73.0, 'date': '2010-01-06', 'id': 4, 'prcp': 0.0, 'station': 'USC00519397'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x000000000A03B198>, 'tobs': 70.0, 'date': '2010-01-07', 'id

In [14]:
engine.execute('SELECT * FROM measurement LIMIT 5').fetchall()

[(0, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (1, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (2, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (3, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (4, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [15]:
engine.execute('SELECT * FROM station').fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999998, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [16]:
session.close()